<a href="https://colab.research.google.com/github/PrinceWallyy/CV_DeepLearning/blob/master/ACML_Assignment_2_Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weights & Biases

In [ ]:
!pip install wandb -q

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

wandb: Currently logged in as: princewallyy (use `wandb login --relogin` to force relogin)


True

# Preprocess

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.datasets import cifar10
from keras.utils import np_utils

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
(x_train, _), (x_test, _) = cifar10.load_data()
data = np.vstack((x_train, x_test))
data = data / 255
(x_train, x_val, x_test) = np.vsplit(data, np.array([48000, 54000]))
assert x_train.shape == (48000, 32, 32, 3)
assert x_val.shape == (6000, 32, 32, 3)
assert x_test.shape == (6000, 32, 32, 3)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

# Sweep

In [ ]:
sweep_config = {
  "name" : "CAE_Random_Optimization",
  "method" : "random",
  "metric":{
    "name": "val_loss",
    "goal": "minimize"
  },
  "parameters" : {
    "kernel" :{
      "values" :[1, 3, 5, 7]
    },
    "stride" :{
      "values" :[1]
    },
    "padding" :{
      "values" :['same']
    },
    "layer_1_filter" :{
      "values" :[8, 12, 16, 32]
    },
    "layer_2_filter" :{
      "values" :[12, 16, 32, 64]
    },
    "layer_3_filter" :{
      "values" :[16, 32, 64, 128]
    },
    "layer_4_filter" :{
      "values" :[12, 16, 32, 64]
    }
  }
}

sweep_id = wandb.sweep(sweep_config, project="Assignment 2")

Create sweep with ID: h4ud3tou
Sweep URL: https://wandb.ai/princewallyy/Assignment%202/sweeps/h4ud3tou


# Convolutional AutoEncoder

In [ ]:
input_shape = (32, 32, 3)

In [ ]:
def createModel(config):
  model = models.Sequential()
  model.add(layers.Conv2D(filters=config.layer_1_filter, kernel_size=(config.kernel, config.kernel), strides=config.stride,
                          padding=config.padding, activation='relu', input_shape=input_shape))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))

  model.add(layers.Conv2D(filters=config.layer_2_filter, kernel_size=(config.kernel, config.kernel), strides=config.stride,
                          padding=config.padding, activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))

  model.add(layers.Conv2D(filters=config.layer_3_filter, kernel_size=(config.kernel, config.kernel), strides=config.stride,
                          padding=config.padding, activation='relu'))
  model.add(layers.UpSampling2D(size=(2, 2)))

  model.add(layers.Conv2D(filters=config.layer_4_filter, kernel_size=(config.kernel, config.kernel), strides=config.stride,
                          padding=config.padding, activation='relu'))
  model.add(layers.UpSampling2D(size=(2, 2)))

  model.add(layers.Conv2D(3, (3, 3), padding='same', activation='sigmoid'))

  return model

# Training

In [ ]:
def train():
  with wandb.init() as run:
    config = wandb.config
    model = createModel(config)

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    model.fit(x_train, x_train, epochs=10, validation_data=(x_val, x_val), callbacks=[WandbCallback()])

    loss, accuracy = model.evaluate(x_test, x_test)
    wandb.log({'Test Error Rate': round((1 - accuracy) * 100, 2)})

In [ ]:
wandb.agent(sweep_id, function=train, count=10, project="Assignment 2")

wandb: Agent Starting Run: 8lkxoshf with config:
wandb: 	kernel: 7
wandb: 	layer_1_filter: 16
wandb: 	layer_2_filter: 16
wandb: 	layer_3_filter: 16
wandb: 	layer_4_filter: 12
wandb: 	padding: same
wandb: 	stride: 1


Epoch 1/10
1500/1500 [==============================] - 43s 9ms/step - loss: 0.0110 - accuracy: 0.6720 - val_loss: 0.0066 - val_accuracy: 0.7478
Epoch 2/10
1500/1500 [==============================] - 13s 9ms/step - loss: 0.0061 - accuracy: 0.7524 - val_loss: 0.0055 - val_accuracy: 0.7423
Epoch 3/10
1500/1500 [==============================] - 13s 9ms/step - loss: 0.0053 - accuracy: 0.7681 - val_loss: 0.0049 - val_accuracy: 0.7657
Epoch 4/10
1500/1500 [==============================] - 13s 9ms/step - loss: 0.0048 - accuracy: 0.7764 - val_loss: 0.0047 - val_accuracy: 0.7843
Epoch 5/10
1500/1500 [==============================] - 13s 9ms/step - loss: 0.0045 - accuracy: 0.7812 - val_loss: 0.0044 - val_accuracy: 0.8039
Epoch 6/10
1500/1500 [==============================] - 13s 9ms/step - loss: 0.0043 - accuracy: 0.7868 - val_loss: 0.0040 - val_accuracy: 0.7864
Epoch 7/10
1500/1500 [==============================] - 13s 9ms/step - loss: 0.0041 - accuracy: 0.7894 - val_loss: 0.0039 - val_ac

Test Error Rate,▁
accuracy,▁▆▆▇▇█████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▃▂▂▂▁▁▁▁
val_accuracy,▂▁▄▆█▆▇▇▇▇
val_loss,█▆▄▄▃▂▂▂▂▁
Test Error Rate,20.47
accuracy,0.79529
best_epoch,9
best_val_loss,0.00352
epoch,9


wandb: Agent Starting Run: x46rwalj with config:
wandb: 	kernel: 5
wandb: 	layer_1_filter: 32
wandb: 	layer_2_filter: 64
wandb: 	layer_3_filter: 16
wandb: 	layer_4_filter: 32
wandb: 	padding: same
wandb: 	stride: 1


Epoch 1/10
1500/1500 [==============================] - 18s 12ms/step - loss: 0.0078 - accuracy: 0.7299 - val_loss: 0.0045 - val_accuracy: 0.7781
Epoch 2/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.0041 - accuracy: 0.7874 - val_loss: 0.0037 - val_accuracy: 0.7791
Epoch 3/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.0037 - accuracy: 0.7990 - val_loss: 0.0034 - val_accuracy: 0.8041
Epoch 4/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.0034 - accuracy: 0.8038 - val_loss: 0.0032 - val_accuracy: 0.8073
Epoch 5/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.0031 - accuracy: 0.8068 - val_loss: 0.0029 - val_accuracy: 0.7890
Epoch 6/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.0029 - accuracy: 0.8085 - val_loss: 0.0027 - val_accuracy: 0.8064
Epoch 7/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.0027 - accuracy: 0.8101 - val_loss: 0.0026 -

...

Test Error Rate,▁
accuracy,▁▅▆▇▇▇████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▂▂▂▁▁▁▁
val_accuracy,▁▃▅▄▅▇▇█▇▆
val_loss,█▅▄▃▃▂▂▁▁▁
Test Error Rate,20.34
accuracy,0.80773
best_epoch,9
best_val_loss,0.00313
epoch,9


wandb: Agent Starting Run: 8oadxbtl with config:
wandb: 	kernel: 7
wandb: 	layer_1_filter: 8
wandb: 	layer_2_filter: 16
wandb: 	layer_3_filter: 16
wandb: 	layer_4_filter: 12
wandb: 	padding: same
wandb: 	stride: 1


Epoch 1/10
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0120 - accuracy: 0.6108 - val_loss: 0.0070 - val_accuracy: 0.7044
Epoch 2/10
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0065 - accuracy: 0.7277 - val_loss: 0.0058 - val_accuracy: 0.7387
Epoch 3/10
1500/1500 [==============================] - 19s 13ms/step - loss: 0.0057 - accuracy: 0.7519 - val_loss: 0.0064 - val_accuracy: 0.7608
Epoch 4/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.0053 - accuracy: 0.7616 - val_loss: 0.0052 - val_accuracy: 0.7758
Epoch 5/10
1500/1500 [==============================] - 15s 10ms/step - loss: 0.0049 - accuracy: 0.7686 - val_loss: 0.0049 - val_accuracy: 0.7537
Epoch 6/10
1500/1500 [==============================] - 14s 9ms/step - loss: 0.0047 - accuracy: 0.7718 - val_loss: 0.0046 - val_accuracy: 0.7813
Epoch 7/10
1500/1500 [==============================] - 13s 9ms/step - loss: 0.0045 - accuracy: 0.7759 - val_loss: 0.0046 - v

Test Error Rate,▁
accuracy,▁▆▇▇▇█████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▂▁▁▁▁
val_accuracy,▁▄▅▆▅▇▅███
val_loss,█▅▇▄▃▃▃▂▁▁
Test Error Rate,20.5
accuracy,0.78273
best_epoch,9
best_val_loss,0.00385
epoch,9


wandb: Agent Starting Run: fxorty0g with config:
wandb: 	kernel: 3
wandb: 	layer_1_filter: 12
wandb: 	layer_2_filter: 32
wandb: 	layer_3_filter: 128
wandb: 	layer_4_filter: 12
wandb: 	padding: same
wandb: 	stride: 1
